In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.path import Path
import cv2
from multiprocess import Pool
import pandas

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *
from cell_utilities import *
from learning_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_anchor.txt
File does not exist: 

In [2]:
stack = 'MD589'

In [3]:
contour_df, _ = DataManager.load_annotation_v3(stack=stack)
from annotation_utilities import convert_annotation_v3_original_to_aligned_cropped
contour_df = convert_annotation_v3_original_to_aligned_cropped(contour_df, stack)

'No object named structures in the file'


Annotation has no structures.


In [4]:
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [35]:
# for sec in range(first_sec, last_sec+1):    
# for sec in [219]:
for sec in [201]:

    fn = metadata_cache['sections_to_filenames'][stack][sec]
    if is_invalid(fn):
        continue
    
    neighbors = load_cell_data(what='neighborCellIndices', stack=stack, sec=sec, ext='hdf')
    neighbor_vectors = load_cell_data(what='neighborVectors', stack=stack, sec=sec, ext='hdf')
    radial_indices = load_cell_data(what='neighborRadialHistBins', stack=stack, sec=sec, ext='hdf')
    angular_indices = load_cell_data(what='neighborAngularHistBins', stack=stack, sec=sec, ext='hdf')
    
#     neighbor_info = load_cell_classifier_data(what='neighbor_info', stack=stack, sec=sec, ext='pkl')
#     neighbors = neighbor_info['neighbors']
#     radial_indices = neighbor_info['radial_indices']
#     angular_indices = neighbor_info['angular_indices']

    # sample_indices = np.random.choice(neighbors.keys(), 10, replace=False)

    # context_histograms = {cell_ind: np.bincount([n_angular*ri+ai 
    #                                  for ri, ai in zip(radial_indices[cell_ind], angular_indices[cell_ind])],
    #                                   minlength=n_angular*n_radial)
    #                       for cell_ind in sample_indices}

    # plot_histograms(context_histograms, bins=range(n_angular*n_radial), 
    #                xlabel='Radial-angular bin',
    #                ylabel='Number of cells',
    #                suptitle='Neighbor placement histogram for random cells')

    cell_orientations = load_cell_data('orientation', stack, sec)
    cell_orientations = np.array(map(normalize_angle, cell_orientations))

    cell_centroids = load_cell_data('centroid', stack, sec)
    cell_numbers = cell_centroids.shape[0]

    image = load_cell_data('image', stack=stack, sec=sec)

    # For visualization

    cell_contours = load_cell_data('contours', stack=stack, sec=sec)
    n_cells = len(cell_contours)

    cell_mask_centers = load_cell_data('mask_center', stack=stack, sec=sec)
    cell_contours_global = {i: cell_centroids[i].astype(np.int) - cell_mask_centers[i] + cell_contours[i]
                            for i in range(cell_numbers)}

    ######################################

#     cells = load_cell_data('cells_aligned_padded', stack, sec)
#     cell_sizes = np.sum(cells.reshape((cells.shape[0], -1)), axis=1)
    cell_sizes = load_cell_data('cellSizes', ext='bp', stack=stack, sec=sec)

    ######################################

#     large_cell_threshold = 163
#     large_cell_indices = np.where(cell_sizes > large_cell_threshold)[0]

    large_cell_indices = load_cell_data('largeCellIndices', stack=stack, sec=sec, ext='bp')
    large_cell_indices_list = large_cell_indices.tolist()

    contours = {cnt['name']: cnt['vertices'] for cnt_id, cnt in contour_df[contour_df['section'] == sec].iterrows()}

    #######################################

    cell_graph_output_dir = create_if_not_exists(os.path.join(CELLS_ROOTDIR, 'cell_graphs', fn))

    #######################################

    large_cell_centroids = cell_centroids[large_cell_indices]
    

    for name_u, cnt in contours.iteritems():

        if name_u not in ['Tz']:
            continue

        # Get large cells

        large_cell_is_inside = Path(cnt.astype(np.int)).contains_points(large_cell_centroids)
        large_cell_indices_inside = large_cell_indices[large_cell_is_inside]

        if len(large_cell_indices_inside) == 0:
            continue
        
        print '%d large cells are identified in %s.' % (len(large_cell_indices_inside), name_u)

        # Small cells

        small_cell_indices = np.array(list(set(range(cell_numbers)) - set(large_cell_indices.tolist())))
        small_cell_centroids = cell_centroids[small_cell_indices]
        small_cell_is_inside = Path(cnt.astype(np.int)).contains_points(small_cell_centroids)
        small_cell_indices_inside = small_cell_indices[small_cell_is_inside]

        print '%d small cells are identified in %s.' % (len(small_cell_indices_inside), name_u)

        #############
        # Visualize #
        #############

        viz = image.copy()
        
        # Draw large cell boundaries
        for i in large_cell_indices_inside:
            cv2.polylines(viz, [cell_contours_global[i].astype(np.int)], isClosed=True, color=(255,0,0), thickness=1)

        # Draw small cell boundaries
        for i in small_cell_indices_inside:
            cv2.polylines(viz, [cell_contours_global[i].astype(np.int)], isClosed=True, color=(0,255,0), thickness=1)

        # Draw structure boundary
        cv2.polylines(viz, [cnt.astype(np.int)], isClosed=True, color=(0,0,255), thickness=1)

        # Visualize radial-angular histogram of selected cells
        random_large_cell_indices = large_cell_indices_inside
    #     random_large_cell_indices = np.random.choice(large_cell_indices_inside, 2, replace=False)

        for source in random_large_cell_indices:

            cv2.circle(viz, tuple(cell_centroids[source].astype(np.int)), 6, color=(255,0,0), thickness=-1)

            neighbor_distances = np.sqrt(np.sum(neighbor_vectors[large_cell_indices_list.index(source)]**2, axis=1))

            c = 0
            for i in np.array(neighbors[large_cell_indices_list.index(source)])[np.argsort(neighbor_distances)]:
    #         for i in neighbors[source]:

                # Draw link to large cells
                if i in large_cell_indices_inside:

                    cv2.circle(viz, tuple(cell_centroids[i].astype(np.int)), 3, color=(255,0,0), thickness=-1)

                    cv2.line(viz, tuple(cell_centroids[source].astype(np.int)), 
                         tuple(cell_centroids[i].astype(np.int)), 
                         color=(255,0,0), thickness=3)
                    
                    c += 1
                    if c == 2:
                        break

                # Draw link to small cells
                if i in small_cell_indices_inside:
                    
                    cv2.circle(viz, tuple(cell_centroids[i].astype(np.int)), 3, color=(0,255,0), thickness=-1)            

                    cv2.line(viz, tuple(cell_centroids[source].astype(np.int)), 
                             tuple(cell_centroids[i].astype(np.int)), 
                             color=(0,0,0), thickness=1)

        # Find crop box

        margin = 50
        roi_xmin, roi_ymin = cnt.astype(np.int).min(axis=0) - margin
        roi_xmax, roi_ymax = cnt.astype(np.int).max(axis=0) + margin
        print roi_xmin, roi_xmax, roi_ymin, roi_ymax

#         imsave(os.path.join(cell_graph_output_dir, '%(fn)s_%(name_u)s_largeAndSmallGraph.png' % dict(fn=fn, name_u=name_u)), 
#                viz[roi_ymin:roi_ymax+1, roi_xmin:roi_xmax+1])

#     roi_xmin = 10621
#     roi_xmax = 12978
#     roi_ymin = 8254
#     roi_ymax = 9698

#     imsave(os.path.join(cell_graph_output_dir, 'temp.png'), 
#            viz[roi_ymin:roi_ymax+1, roi_xmin:roi_xmax+1])


88 large cells are identified in Tz.
75 small cells are identified in Tz.
4918 5794 9981 10649


In [36]:
display_image(viz[roi_ymin:roi_ymax+1, roi_xmin:roi_xmax+1])

/shared/MouseBrainAtlas/cells/tmp.jpg

# Histograms

In [ ]:
plot_histograms({name_u: h/float(h.sum()) for name_u, h in large_cell_size_histogram_all_regions.iteritems()},
                bins=size_bins[:-1],
               xlabel='Size (pixels)', 
                ylabel='Number of cells',
               suptitle='Histogram of LARGE cell size')

In [ ]:
plot_histograms({name_u: h/float(h.sum()) for name_u, h in all_cell_size_weighted_histogram_all_regions.iteritems()},
                bins=size_bins[:-1],
               xlabel='Size (pixels)', 
                ylabel='Number of cells',
               suptitle='WEIGHTED histogram of cell size')

In [ ]:
plot_histograms({name_u: h/float(h.sum()) for name_u, h in all_cell_size_histogram_all_regions.iteritems()},
                bins=size_bins[:-1],
               xlabel='Size (pixels)', 
                ylabel='Number of cells',
               suptitle='Histogram of cell size')

In [ ]:
plot_histograms({name_u: h/float(h.sum()) for name_u, h in large_cell_orientation_histogram_all_regions.iteritems()},
                bins=orientation_bins[:-1],
               xlabel='Orientation (radian)', 
                ylabel='Number of cells',
               suptitle='Histogram of LARGE cell orientation')

In [ ]:
plot_histograms({name_u: h/float(h.sum()) for name_u, h in neighbor_distance_histogram_all_regions.iteritems()},
                bins=edge_length_bins[:-1],
               xlabel='Neighbor distance (pixel)', 
                ylabel='Number of cells',
               suptitle='Histogram of neighbor distance')

In [ ]:
plot_histograms({name_u: h/float(h.sum()) for name_u, h in neighbor_direction_histogram_all_regions.iteritems()},
                bins=edge_direction_bins[:-1],
               xlabel='Neighbor direction (radian)', 
                ylabel='Number of cells',
               suptitle='Histogram of neighbor direction')

In [ ]:
plot_histograms({name_u: h/float(h.sum()) for name_u, h in large_cell_graph_neighbor_distance_histogram_all_regions.iteritems()},
                bins=edge_length_bins[:-1],
               xlabel='Neighbor distance (pixel)', 
                ylabel='Number of cells',
               suptitle='Histogram of neighbor distance for large cell graphs')

In [ ]:
plot_histograms({name_u: h/float(h.sum()) for name_u, h in large_cell_graph_neighbor_direction_histogram_all_regions.iteritems()},
                bins=edge_direction_bins[:-1],
               xlabel='Neighbor direction (radian)', 
                ylabel='Number of cells',
               suptitle='Histogram of neighbor direction for large cell graphs')

In [ ]:
plot_histograms({name_u: h/float(h.sum()) for name_u, h in large_small_cell_graph_neighbor_distance_histogram_all_regions.iteritems()},
                bins=edge_length_bins[:-1],
               xlabel='Neighbor distance (pixel)', 
                ylabel='Number of cells',
               suptitle='Histogram of neighbor distance for large-small cell graphs')

In [ ]:
plot_histograms({name_u: h/float(h.sum()) for name_u, h in large_small_cell_graph_neighbor_direction_histogram_all_regions.iteritems()},
                bins=edge_direction_bins[:-1],
               xlabel='Neighbor direction (radian)', 
                ylabel='Number of cells',
               suptitle='Histogram of neighbor direction for large-small cell graphs')